<a href="https://colab.research.google.com/github/VNagaHarshitha/SentimentAnalysisProject/blob/main/sentimentanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
from textblob import TextBlob

# Your API key
API_KEY = "pub_5e145b042e4f4616a29413e6c565817d"
BASE_URL = "https://newsdata.io/api/1/news"

# Function to fetch news from API with pagination
def fetch_news(query="business", language="en", max_articles=100):
    all_articles = []
    next_page = None

    while len(all_articles) < max_articles:
        params = {
            "apikey": API_KEY,
            "q": query,
            "language": language,
        }
        if next_page:
            params["page"] = next_page

        response = requests.get(BASE_URL, params=params)
        if response.status_code == 200:
            data = response.json()
            if data.get("status") == "success":
                articles = data.get("results", [])
                all_articles.extend(articles)

                # Check if there's a next page
                next_page = data.get("nextPage")
                if not next_page:
                    break  # No more pages available
            else:
                print("API error:", data)
                break
        else:
            print("Error:", response.status_code, response.text)
            break

    return all_articles[:max_articles]  # Limit to requested number


# Sentiment Analysis Function
def analyze_sentiment(text):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    if polarity > 0:
        return "Positive"
    elif polarity < 0:
        return "Negative"
    else:
        return "Neutral"


# Predict Market/Business Trend
def predict_market(sentiments):
    pos = sentiments.count("Positive")
    neg = sentiments.count("Negative")
    neu = sentiments.count("Neutral")

    if pos > neg:
        return "Market Outlook: Bullish (Positive trend)"
    elif neg > pos:
        return "Market Outlook: Bearish (Negative trend)"
    else:
        return "Market Outlook: Neutral / Uncertain"


# Main execution
if __name__ == "__main__":
    articles = fetch_news(query="stock market", max_articles=120)  # Get ~120
    if articles:
        news_list = []
        sentiments = []

        for article in articles:
            title = article.get("title", "")
            description = article.get("description", "")
            content = (title or "") + " " + (description or "")

            if not content.strip():
                continue

            sentiment = analyze_sentiment(content)
            sentiments.append(sentiment)

            news_list.append({
                "Title": title,
                "Description": description,
                "Sentiment": sentiment
            })

        if news_list:
            df = pd.DataFrame(news_list)
            print(df)

            # 🔽 Save results to CSV
            df.to_csv("news_sentiment_analysis.csv", index=False)
            print("\n✅ Data exported to news_sentiment_analysis.csv")
            print(f"✅ Total Articles Analyzed: {len(news_list)}")

            print("\n📈", predict_market(sentiments))
        else:
            print("No analyzable articles found.")
    else:
        print("No news data found.")


                                                 Title  \
0              Taurus Horoscope Today, August 30, 2025   
1    Crypto Shake-Up Ahead: BTC Flows to ETH as Ana...   
2    Gamuda racks up RM21.7bil contracts so far in ...   
3    Numerology, August 30, 2025: Fiery 5, Tender 2...   
4    Land & General Berhad's (KLSE:L&G) Shareholder...   
..                                                 ...   
115  Spirit Airlines seeks second bankruptcy as dwi...   
116         S&P 500 ends lower as Dell and Nvidia drop   
117  Why Lucid Stock Skidded to a More Than 4% Loss...   
118  Most Trump tariffs are not legal, US appeals c...   
119  US appeals court finds Trump's tariffs illegal...   

                                           Description Sentiment  
0    Taurus Daily Horoscope, August 30, 2025: Finan...  Positive  
1    Moonshot Magax Presale shines as Bitcoin flows...  Positive  
2    KUALA LUMPUR: Gamuda Bhd is powering through 2...  Positive  
3    Numerology, August 30, 2025: N